In [4]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

# Graph AutoEncoder GAE

## Load the data

In [5]:
## T.NOrmalizaeFeatures() ? !
## none have the label (Autoencoder is unsupervised learning)

dataset = Planetoid('\..', 'CiteSeer', transform=T.NormalizeFeatures())
dataset.data

Processing...
Done!


Data(edge_index=[2, 9104], test_mask=[3327], train_mask=[3327], val_mask=[3327], x=[3327, 3703], y=[3327])

In [7]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = None
data

Data(edge_index=[2, 9104], x=[3327, 3703], y=[3327])

In [9]:
data

Data(edge_index=[2, 9104], x=[3327, 3703], y=[3327])

In [10]:
data = train_test_split_edges(data)

In [11]:
data

Data(test_neg_edge_index=[2, 455], test_pos_edge_index=[2, 455], train_neg_adj_mask=[3327, 3327], train_pos_edge_index=[2, 7740], val_neg_edge_index=[2, 227], val_pos_edge_index=[2, 227], x=[3327, 3703], y=[3327])

## Define the Encoder

In [17]:
# cached ? what argument ??

class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

## Define the Autoencoder

In [14]:
from torch_geometric.nn import GAE

In [46]:
# paramter
out_channels = 2
num_features = dataset.num_features ##
epochs = 5000

model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# initialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [21]:
# useful keyword
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_module',
 'apply',


### ## check the results with wandb

In [47]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    # if args.variational:
    # loss = loss + (1 / data.num_nodes) * model.kl_loss() # 쿨랙라이블러 발산을 활용한 distribution correction 
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
        
    return model.test(z, pos_edge_index, neg_edge_index)

In [48]:
import wandb
wandb.init()
## pos edge index vs. neg edge index 는 무엇을 의미하는가 ? 

for epoch in range(1, epochs + 1):
    loss = train()
    
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    wandb.log({
        'test epoch' : epoch,
        'test accuracy' : auc,
        'test ap' : ap
    })
    
    print(f'Epoch: {epoch:03d}, AUC: {auc:4f}, AP: {ap:.4f}')

test epoch,10000
test accuracy,0.7682
test ap,0.78114
_runtime,169
_timestamp,1629955137
_step,9999


test epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test accuracy,██▇▇▇▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
test ap,████▇▇▇▇▇▆▆▆▆▆▆▆▆▅▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


Epoch: 001, AUC: 0.642785, AP: 0.6633
Epoch: 002, AUC: 0.648612, AP: 0.6689
Epoch: 003, AUC: 0.651016, AP: 0.6727
Epoch: 004, AUC: 0.651960, AP: 0.6749
Epoch: 005, AUC: 0.652332, AP: 0.6764
Epoch: 006, AUC: 0.652515, AP: 0.6773
Epoch: 007, AUC: 0.653006, AP: 0.6783
Epoch: 008, AUC: 0.653042, AP: 0.6793
Epoch: 009, AUC: 0.653337, AP: 0.6808
Epoch: 010, AUC: 0.653588, AP: 0.6824
Epoch: 011, AUC: 0.653254, AP: 0.6842
Epoch: 012, AUC: 0.652443, AP: 0.6859
Epoch: 013, AUC: 0.652064, AP: 0.6886
Epoch: 014, AUC: 0.651467, AP: 0.6915
Epoch: 015, AUC: 0.649825, AP: 0.6936
Epoch: 016, AUC: 0.648050, AP: 0.6964
Epoch: 017, AUC: 0.645333, AP: 0.6979
Epoch: 018, AUC: 0.643492, AP: 0.6994
Epoch: 019, AUC: 0.641551, AP: 0.7011
Epoch: 020, AUC: 0.640024, AP: 0.7028
Epoch: 021, AUC: 0.638575, AP: 0.7033
Epoch: 022, AUC: 0.637655, AP: 0.7039
Epoch: 023, AUC: 0.636720, AP: 0.7039
Epoch: 024, AUC: 0.636068, AP: 0.7042
Epoch: 025, AUC: 0.635773, AP: 0.7047
Epoch: 026, AUC: 0.635590, AP: 0.7051
Epoch: 027, 

Epoch: 219, AUC: 0.783488, AP: 0.7839
Epoch: 220, AUC: 0.783178, AP: 0.7837
Epoch: 221, AUC: 0.782744, AP: 0.7833
Epoch: 222, AUC: 0.781884, AP: 0.7825
Epoch: 223, AUC: 0.782357, AP: 0.7834
Epoch: 224, AUC: 0.782874, AP: 0.7836
Epoch: 225, AUC: 0.782633, AP: 0.7829
Epoch: 226, AUC: 0.782116, AP: 0.7830
Epoch: 227, AUC: 0.781183, AP: 0.7827
Epoch: 228, AUC: 0.780802, AP: 0.7824
Epoch: 229, AUC: 0.781478, AP: 0.7825
Epoch: 230, AUC: 0.782659, AP: 0.7833
Epoch: 231, AUC: 0.782495, AP: 0.7831
Epoch: 232, AUC: 0.781671, AP: 0.7831
Epoch: 233, AUC: 0.781208, AP: 0.7827
Epoch: 234, AUC: 0.780628, AP: 0.7819
Epoch: 235, AUC: 0.781256, AP: 0.7831
Epoch: 236, AUC: 0.782321, AP: 0.7837
Epoch: 237, AUC: 0.782364, AP: 0.7823
Epoch: 238, AUC: 0.782094, AP: 0.7819
Epoch: 239, AUC: 0.782809, AP: 0.7847
Epoch: 240, AUC: 0.781270, AP: 0.7830
Epoch: 241, AUC: 0.781217, AP: 0.7834
Epoch: 242, AUC: 0.781761, AP: 0.7839
Epoch: 243, AUC: 0.781966, AP: 0.7834
Epoch: 244, AUC: 0.782031, AP: 0.7826
Epoch: 245, 

Epoch: 439, AUC: 0.771525, AP: 0.7735
Epoch: 440, AUC: 0.771330, AP: 0.7736
Epoch: 441, AUC: 0.770501, AP: 0.7725
Epoch: 442, AUC: 0.769675, AP: 0.7715
Epoch: 443, AUC: 0.769825, AP: 0.7719
Epoch: 444, AUC: 0.770342, AP: 0.7723
Epoch: 445, AUC: 0.771267, AP: 0.7733
Epoch: 446, AUC: 0.771441, AP: 0.7732
Epoch: 447, AUC: 0.771189, AP: 0.7732
Epoch: 448, AUC: 0.770641, AP: 0.7722
Epoch: 449, AUC: 0.770392, AP: 0.7722
Epoch: 450, AUC: 0.770868, AP: 0.7726
Epoch: 451, AUC: 0.771525, AP: 0.7734
Epoch: 452, AUC: 0.771255, AP: 0.7733
Epoch: 453, AUC: 0.771088, AP: 0.7733
Epoch: 454, AUC: 0.770475, AP: 0.7729
Epoch: 455, AUC: 0.770277, AP: 0.7729
Epoch: 456, AUC: 0.770504, AP: 0.7731
Epoch: 457, AUC: 0.770854, AP: 0.7732
Epoch: 458, AUC: 0.770772, AP: 0.7730
Epoch: 459, AUC: 0.770178, AP: 0.7730
Epoch: 460, AUC: 0.769554, AP: 0.7727
Epoch: 461, AUC: 0.769467, AP: 0.7726
Epoch: 462, AUC: 0.768965, AP: 0.7721
Epoch: 463, AUC: 0.768835, AP: 0.7720
Epoch: 464, AUC: 0.768774, AP: 0.7719
Epoch: 465, 

...

Epoch: 3963, AUC: 0.800814, AP: 0.8170
Epoch: 3964, AUC: 0.800553, AP: 0.8171
Epoch: 3965, AUC: 0.799679, AP: 0.8171
Epoch: 3966, AUC: 0.799445, AP: 0.8173
Epoch: 3967, AUC: 0.799181, AP: 0.8163
Epoch: 3968, AUC: 0.799469, AP: 0.8160
Epoch: 3969, AUC: 0.799123, AP: 0.8153
Epoch: 3970, AUC: 0.798993, AP: 0.8152
Epoch: 3971, AUC: 0.798940, AP: 0.8152
Epoch: 3972, AUC: 0.799285, AP: 0.8161
Epoch: 3973, AUC: 0.799184, AP: 0.8159
Epoch: 3974, AUC: 0.799256, AP: 0.8152
Epoch: 3975, AUC: 0.799077, AP: 0.8148
Epoch: 3976, AUC: 0.798648, AP: 0.8140
Epoch: 3977, AUC: 0.798802, AP: 0.8146
Epoch: 3978, AUC: 0.798701, AP: 0.8152
Epoch: 3979, AUC: 0.798573, AP: 0.8156
Epoch: 3980, AUC: 0.798413, AP: 0.8154
Epoch: 3981, AUC: 0.798387, AP: 0.8150
Epoch: 3982, AUC: 0.798237, AP: 0.8146
Epoch: 3983, AUC: 0.797778, AP: 0.8141
Epoch: 3984, AUC: 0.797609, AP: 0.8141
Epoch: 3985, AUC: 0.797778, AP: 0.8145
Epoch: 3986, AUC: 0.797843, AP: 0.8153
Epoch: 3987, AUC: 0.797524, AP: 0.8153
Epoch: 3988, AUC: 0.79741

Epoch: 4180, AUC: 0.795636, AP: 0.8140
Epoch: 4181, AUC: 0.796302, AP: 0.8144
Epoch: 4182, AUC: 0.796829, AP: 0.8147
Epoch: 4183, AUC: 0.797421, AP: 0.8151
Epoch: 4184, AUC: 0.797322, AP: 0.8152
Epoch: 4185, AUC: 0.796788, AP: 0.8151
Epoch: 4186, AUC: 0.795943, AP: 0.8148
Epoch: 4187, AUC: 0.796469, AP: 0.8151
Epoch: 4188, AUC: 0.796720, AP: 0.8149
Epoch: 4189, AUC: 0.796331, AP: 0.8145
Epoch: 4190, AUC: 0.795684, AP: 0.8141
Epoch: 4191, AUC: 0.795542, AP: 0.8143
Epoch: 4192, AUC: 0.794764, AP: 0.8139
Epoch: 4193, AUC: 0.794762, AP: 0.8139
Epoch: 4194, AUC: 0.795199, AP: 0.8140
Epoch: 4195, AUC: 0.795085, AP: 0.8139
Epoch: 4196, AUC: 0.795039, AP: 0.8137
Epoch: 4197, AUC: 0.794947, AP: 0.8135
Epoch: 4198, AUC: 0.794904, AP: 0.8134
Epoch: 4199, AUC: 0.795329, AP: 0.8138
Epoch: 4200, AUC: 0.795645, AP: 0.8141
Epoch: 4201, AUC: 0.795887, AP: 0.8142
Epoch: 4202, AUC: 0.796097, AP: 0.8144
Epoch: 4203, AUC: 0.796256, AP: 0.8144
Epoch: 4204, AUC: 0.796025, AP: 0.8142
Epoch: 4205, AUC: 0.79556

Epoch: 4397, AUC: 0.800060, AP: 0.8165
Epoch: 4398, AUC: 0.800205, AP: 0.8164
Epoch: 4399, AUC: 0.800324, AP: 0.8165
Epoch: 4400, AUC: 0.800341, AP: 0.8167
Epoch: 4401, AUC: 0.800200, AP: 0.8166
Epoch: 4402, AUC: 0.800476, AP: 0.8167
Epoch: 4403, AUC: 0.800683, AP: 0.8169
Epoch: 4404, AUC: 0.801227, AP: 0.8172
Epoch: 4405, AUC: 0.801205, AP: 0.8171
Epoch: 4406, AUC: 0.801111, AP: 0.8170
Epoch: 4407, AUC: 0.800647, AP: 0.8167
Epoch: 4408, AUC: 0.800604, AP: 0.8165
Epoch: 4409, AUC: 0.800394, AP: 0.8162
Epoch: 4410, AUC: 0.799628, AP: 0.8155
Epoch: 4411, AUC: 0.799283, AP: 0.8151
Epoch: 4412, AUC: 0.799092, AP: 0.8151
Epoch: 4413, AUC: 0.799128, AP: 0.8154
Epoch: 4414, AUC: 0.798399, AP: 0.8154
Epoch: 4415, AUC: 0.797819, AP: 0.8151
Epoch: 4416, AUC: 0.797751, AP: 0.8147
Epoch: 4417, AUC: 0.798235, AP: 0.8146
Epoch: 4418, AUC: 0.797954, AP: 0.8143
Epoch: 4419, AUC: 0.797370, AP: 0.8139
Epoch: 4420, AUC: 0.797336, AP: 0.8141
Epoch: 4421, AUC: 0.796433, AP: 0.8141
Epoch: 4422, AUC: 0.79553

Epoch: 4620, AUC: 0.797409, AP: 0.8141
Epoch: 4621, AUC: 0.797800, AP: 0.8140
Epoch: 4622, AUC: 0.797720, AP: 0.8138
Epoch: 4623, AUC: 0.797662, AP: 0.8140
Epoch: 4624, AUC: 0.796735, AP: 0.8139
Epoch: 4625, AUC: 0.796988, AP: 0.8146
Epoch: 4626, AUC: 0.796496, AP: 0.8142
Epoch: 4627, AUC: 0.796578, AP: 0.8143
Epoch: 4628, AUC: 0.797278, AP: 0.8141
Epoch: 4629, AUC: 0.797636, AP: 0.8136
Epoch: 4630, AUC: 0.797520, AP: 0.8137
Epoch: 4631, AUC: 0.797462, AP: 0.8139
Epoch: 4632, AUC: 0.797599, AP: 0.8150
Epoch: 4633, AUC: 0.797283, AP: 0.8149
Epoch: 4634, AUC: 0.797348, AP: 0.8149
Epoch: 4635, AUC: 0.797329, AP: 0.8149
Epoch: 4636, AUC: 0.797396, AP: 0.8147
Epoch: 4637, AUC: 0.797150, AP: 0.8146
Epoch: 4638, AUC: 0.797090, AP: 0.8143
Epoch: 4639, AUC: 0.796964, AP: 0.8141
Epoch: 4640, AUC: 0.797068, AP: 0.8139
Epoch: 4641, AUC: 0.797133, AP: 0.8138
Epoch: 4642, AUC: 0.797428, AP: 0.8143
Epoch: 4643, AUC: 0.797445, AP: 0.8144
Epoch: 4644, AUC: 0.797512, AP: 0.8145
Epoch: 4645, AUC: 0.79732

Epoch: 4839, AUC: 0.798391, AP: 0.8178
Epoch: 4840, AUC: 0.798483, AP: 0.8180
Epoch: 4841, AUC: 0.799063, AP: 0.8179
Epoch: 4842, AUC: 0.799060, AP: 0.8172
Epoch: 4843, AUC: 0.798945, AP: 0.8167
Epoch: 4844, AUC: 0.798761, AP: 0.8168
Epoch: 4845, AUC: 0.798785, AP: 0.8168
Epoch: 4846, AUC: 0.798587, AP: 0.8170
Epoch: 4847, AUC: 0.797809, AP: 0.8167
Epoch: 4848, AUC: 0.797785, AP: 0.8164
Epoch: 4849, AUC: 0.797974, AP: 0.8161
Epoch: 4850, AUC: 0.797802, AP: 0.8159
Epoch: 4851, AUC: 0.797358, AP: 0.8154
Epoch: 4852, AUC: 0.797230, AP: 0.8156
Epoch: 4853, AUC: 0.797387, AP: 0.8163
Epoch: 4854, AUC: 0.797121, AP: 0.8165
Epoch: 4855, AUC: 0.796947, AP: 0.8165
Epoch: 4856, AUC: 0.797092, AP: 0.8163
Epoch: 4857, AUC: 0.797167, AP: 0.8160
Epoch: 4858, AUC: 0.796880, AP: 0.8154
Epoch: 4859, AUC: 0.796761, AP: 0.8154
Epoch: 4860, AUC: 0.796814, AP: 0.8156
Epoch: 4861, AUC: 0.797785, AP: 0.8167
Epoch: 4862, AUC: 0.797899, AP: 0.8171
Epoch: 4863, AUC: 0.797537, AP: 0.8168
Epoch: 4864, AUC: 0.79746

In [32]:
Z = model.encode(x, train_pos_edge_index)
Z

tensor([[ 0.2753, -0.3158],
        [-0.7813,  0.9248],
        [ 0.6444, -0.7434],
        ...,
        [-0.0554,  0.0861],
        [ 0.5674, -0.6514],
        [ 0.7216, -0.8948]], device='cuda:0', grad_fn=<AddBackward0>)

In [33]:
Z.size()

torch.Size([3327, 2])

### check the results with tensorboard

In [34]:
from torch.utils.tensorboard import SummaryWriter

In [36]:
# paramter
out_channels = 2
num_features = dataset.num_features ##
epochs = 10000

model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# initialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [40]:
writer = SummaryWriter('runs/GAE_experiemnt_' + '2d_100_epochs') 

In [43]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print(f'Epoch: {epoch:03d}, AUC: {auc:.4f}, AP: {ap:.4f}')
          
    writer.add_scalar('auc train', auc,epoch) # viz your experiment
    writer.add_scalar('ap train', ap,epoch) #

Epoch: 001, AUC: 0.5564, AP: 0.5484
Epoch: 002, AUC: 0.5496, AP: 0.5453
Epoch: 003, AUC: 0.5439, AP: 0.5430
Epoch: 004, AUC: 0.5396, AP: 0.5414
Epoch: 005, AUC: 0.5371, AP: 0.5415
Epoch: 006, AUC: 0.5353, AP: 0.5411
Epoch: 007, AUC: 0.5342, AP: 0.5413
Epoch: 008, AUC: 0.5340, AP: 0.5423
Epoch: 009, AUC: 0.5345, AP: 0.5431
Epoch: 010, AUC: 0.5360, AP: 0.5450
Epoch: 011, AUC: 0.5382, AP: 0.5468
Epoch: 012, AUC: 0.5404, AP: 0.5496
Epoch: 013, AUC: 0.5426, AP: 0.5521
Epoch: 014, AUC: 0.5449, AP: 0.5562
Epoch: 015, AUC: 0.5471, AP: 0.5590
Epoch: 016, AUC: 0.5495, AP: 0.5620
Epoch: 017, AUC: 0.5519, AP: 0.5650
Epoch: 018, AUC: 0.5546, AP: 0.5685
Epoch: 019, AUC: 0.5572, AP: 0.5727
Epoch: 020, AUC: 0.5598, AP: 0.5765
Epoch: 021, AUC: 0.5620, AP: 0.5793
Epoch: 022, AUC: 0.5643, AP: 0.5826
Epoch: 023, AUC: 0.5662, AP: 0.5863
Epoch: 024, AUC: 0.5687, AP: 0.5899
Epoch: 025, AUC: 0.5705, AP: 0.5924
Epoch: 026, AUC: 0.5726, AP: 0.5954
Epoch: 027, AUC: 0.5744, AP: 0.5971
Epoch: 028, AUC: 0.5762, AP:

Epoch: 235, AUC: 0.6471, AP: 0.6809
Epoch: 236, AUC: 0.6471, AP: 0.6809
Epoch: 237, AUC: 0.6471, AP: 0.6809
Epoch: 238, AUC: 0.6471, AP: 0.6810
Epoch: 239, AUC: 0.6471, AP: 0.6810
Epoch: 240, AUC: 0.6471, AP: 0.6810
Epoch: 241, AUC: 0.6472, AP: 0.6811
Epoch: 242, AUC: 0.6472, AP: 0.6811
Epoch: 243, AUC: 0.6472, AP: 0.6811
Epoch: 244, AUC: 0.6472, AP: 0.6813
Epoch: 245, AUC: 0.6473, AP: 0.6813
Epoch: 246, AUC: 0.6474, AP: 0.6813
Epoch: 247, AUC: 0.6475, AP: 0.6814
Epoch: 248, AUC: 0.6475, AP: 0.6815
Epoch: 249, AUC: 0.6475, AP: 0.6815
Epoch: 250, AUC: 0.6475, AP: 0.6815
Epoch: 251, AUC: 0.6475, AP: 0.6815
Epoch: 252, AUC: 0.6475, AP: 0.6816
Epoch: 253, AUC: 0.6476, AP: 0.6817
Epoch: 254, AUC: 0.6477, AP: 0.6817
Epoch: 255, AUC: 0.6477, AP: 0.6817
Epoch: 256, AUC: 0.6477, AP: 0.6818
Epoch: 257, AUC: 0.6477, AP: 0.6818
Epoch: 258, AUC: 0.6478, AP: 0.6819
Epoch: 259, AUC: 0.6477, AP: 0.6819
Epoch: 260, AUC: 0.6478, AP: 0.6820
Epoch: 261, AUC: 0.6479, AP: 0.6820
Epoch: 262, AUC: 0.6478, AP:

Epoch: 469, AUC: 0.6486, AP: 0.6906
Epoch: 470, AUC: 0.6486, AP: 0.6907
Epoch: 471, AUC: 0.6486, AP: 0.6907
Epoch: 472, AUC: 0.6486, AP: 0.6908
Epoch: 473, AUC: 0.6486, AP: 0.6909
Epoch: 474, AUC: 0.6486, AP: 0.6909
Epoch: 475, AUC: 0.6486, AP: 0.6910
Epoch: 476, AUC: 0.6486, AP: 0.6910
Epoch: 477, AUC: 0.6487, AP: 0.6911
Epoch: 478, AUC: 0.6486, AP: 0.6911
Epoch: 479, AUC: 0.6486, AP: 0.6912
Epoch: 480, AUC: 0.6486, AP: 0.6913
Epoch: 481, AUC: 0.6486, AP: 0.6912
Epoch: 482, AUC: 0.6485, AP: 0.6912
Epoch: 483, AUC: 0.6485, AP: 0.6913
Epoch: 484, AUC: 0.6485, AP: 0.6914
Epoch: 485, AUC: 0.6486, AP: 0.6915
Epoch: 486, AUC: 0.6485, AP: 0.6915
Epoch: 487, AUC: 0.6485, AP: 0.6916
Epoch: 488, AUC: 0.6485, AP: 0.6915
Epoch: 489, AUC: 0.6485, AP: 0.6917
Epoch: 490, AUC: 0.6484, AP: 0.6916
Epoch: 491, AUC: 0.6484, AP: 0.6917
Epoch: 492, AUC: 0.6484, AP: 0.6917
Epoch: 493, AUC: 0.6484, AP: 0.6918
Epoch: 494, AUC: 0.6484, AP: 0.6918
Epoch: 495, AUC: 0.6483, AP: 0.6918
Epoch: 496, AUC: 0.6483, AP:

Epoch: 703, AUC: 0.6439, AP: 0.6979
Epoch: 704, AUC: 0.6439, AP: 0.6980
Epoch: 705, AUC: 0.6439, AP: 0.6980
Epoch: 706, AUC: 0.6439, AP: 0.6980
Epoch: 707, AUC: 0.6439, AP: 0.6980
Epoch: 708, AUC: 0.6439, AP: 0.6981
Epoch: 709, AUC: 0.6438, AP: 0.6981
Epoch: 710, AUC: 0.6439, AP: 0.6981
Epoch: 711, AUC: 0.6438, AP: 0.6982
Epoch: 712, AUC: 0.6438, AP: 0.6982
Epoch: 713, AUC: 0.6438, AP: 0.6982
Epoch: 714, AUC: 0.6438, AP: 0.6983
Epoch: 715, AUC: 0.6438, AP: 0.6983
Epoch: 716, AUC: 0.6438, AP: 0.6983
Epoch: 717, AUC: 0.6438, AP: 0.6983
Epoch: 718, AUC: 0.6438, AP: 0.6984
Epoch: 719, AUC: 0.6438, AP: 0.6984
Epoch: 720, AUC: 0.6437, AP: 0.6984
Epoch: 721, AUC: 0.6437, AP: 0.6984
Epoch: 722, AUC: 0.6437, AP: 0.6984
Epoch: 723, AUC: 0.6437, AP: 0.6984
Epoch: 724, AUC: 0.6437, AP: 0.6985
Epoch: 725, AUC: 0.6436, AP: 0.6984
Epoch: 726, AUC: 0.6436, AP: 0.6985
Epoch: 727, AUC: 0.6436, AP: 0.6985
Epoch: 728, AUC: 0.6436, AP: 0.6985
Epoch: 729, AUC: 0.6436, AP: 0.6985
Epoch: 730, AUC: 0.6435, AP:

Epoch: 937, AUC: 0.6396, AP: 0.7018
Epoch: 938, AUC: 0.6397, AP: 0.7018
Epoch: 939, AUC: 0.6396, AP: 0.7018
Epoch: 940, AUC: 0.6396, AP: 0.7018
Epoch: 941, AUC: 0.6396, AP: 0.7019
Epoch: 942, AUC: 0.6395, AP: 0.7019
Epoch: 943, AUC: 0.6395, AP: 0.7019
Epoch: 944, AUC: 0.6395, AP: 0.7019
Epoch: 945, AUC: 0.6395, AP: 0.7019
Epoch: 946, AUC: 0.6395, AP: 0.7019
Epoch: 947, AUC: 0.6395, AP: 0.7019
Epoch: 948, AUC: 0.6395, AP: 0.7020
Epoch: 949, AUC: 0.6394, AP: 0.7019
Epoch: 950, AUC: 0.6394, AP: 0.7020
Epoch: 951, AUC: 0.6394, AP: 0.7020
Epoch: 952, AUC: 0.6394, AP: 0.7020
Epoch: 953, AUC: 0.6394, AP: 0.7020
Epoch: 954, AUC: 0.6394, AP: 0.7020
Epoch: 955, AUC: 0.6394, AP: 0.7020
Epoch: 956, AUC: 0.6393, AP: 0.7020
Epoch: 957, AUC: 0.6393, AP: 0.7020
Epoch: 958, AUC: 0.6393, AP: 0.7020
Epoch: 959, AUC: 0.6392, AP: 0.7020
Epoch: 960, AUC: 0.6392, AP: 0.7020
Epoch: 961, AUC: 0.6392, AP: 0.7020
Epoch: 962, AUC: 0.6392, AP: 0.7020
Epoch: 963, AUC: 0.6392, AP: 0.7021
Epoch: 964, AUC: 0.6391, AP:

...

Epoch: 8882, AUC: 0.7799, AP: 0.7861
Epoch: 8883, AUC: 0.7799, AP: 0.7861
Epoch: 8884, AUC: 0.7799, AP: 0.7861
Epoch: 8885, AUC: 0.7799, AP: 0.7861
Epoch: 8886, AUC: 0.7799, AP: 0.7861
Epoch: 8887, AUC: 0.7799, AP: 0.7861
Epoch: 8888, AUC: 0.7799, AP: 0.7861
Epoch: 8889, AUC: 0.7799, AP: 0.7861
Epoch: 8890, AUC: 0.7799, AP: 0.7861
Epoch: 8891, AUC: 0.7799, AP: 0.7862
Epoch: 8892, AUC: 0.7799, AP: 0.7862
Epoch: 8893, AUC: 0.7799, AP: 0.7862
Epoch: 8894, AUC: 0.7799, AP: 0.7862
Epoch: 8895, AUC: 0.7799, AP: 0.7861
Epoch: 8896, AUC: 0.7799, AP: 0.7862
Epoch: 8897, AUC: 0.7799, AP: 0.7861
Epoch: 8898, AUC: 0.7799, AP: 0.7861
Epoch: 8899, AUC: 0.7799, AP: 0.7861
Epoch: 8900, AUC: 0.7798, AP: 0.7861
Epoch: 8901, AUC: 0.7798, AP: 0.7860
Epoch: 8902, AUC: 0.7798, AP: 0.7860
Epoch: 8903, AUC: 0.7798, AP: 0.7860
Epoch: 8904, AUC: 0.7798, AP: 0.7860
Epoch: 8905, AUC: 0.7798, AP: 0.7860
Epoch: 8906, AUC: 0.7797, AP: 0.7859
Epoch: 8907, AUC: 0.7797, AP: 0.7859
Epoch: 8908, AUC: 0.7798, AP: 0.7859
E

Epoch: 9113, AUC: 0.7796, AP: 0.7864
Epoch: 9114, AUC: 0.7796, AP: 0.7863
Epoch: 9115, AUC: 0.7796, AP: 0.7863
Epoch: 9116, AUC: 0.7796, AP: 0.7863
Epoch: 9117, AUC: 0.7796, AP: 0.7863
Epoch: 9118, AUC: 0.7796, AP: 0.7863
Epoch: 9119, AUC: 0.7796, AP: 0.7863
Epoch: 9120, AUC: 0.7796, AP: 0.7863
Epoch: 9121, AUC: 0.7796, AP: 0.7863
Epoch: 9122, AUC: 0.7796, AP: 0.7863
Epoch: 9123, AUC: 0.7796, AP: 0.7863
Epoch: 9124, AUC: 0.7796, AP: 0.7863
Epoch: 9125, AUC: 0.7796, AP: 0.7863
Epoch: 9126, AUC: 0.7796, AP: 0.7863
Epoch: 9127, AUC: 0.7796, AP: 0.7863
Epoch: 9128, AUC: 0.7796, AP: 0.7862
Epoch: 9129, AUC: 0.7796, AP: 0.7862
Epoch: 9130, AUC: 0.7796, AP: 0.7861
Epoch: 9131, AUC: 0.7796, AP: 0.7862
Epoch: 9132, AUC: 0.7796, AP: 0.7862
Epoch: 9133, AUC: 0.7797, AP: 0.7861
Epoch: 9134, AUC: 0.7796, AP: 0.7861
Epoch: 9135, AUC: 0.7796, AP: 0.7861
Epoch: 9136, AUC: 0.7796, AP: 0.7861
Epoch: 9137, AUC: 0.7796, AP: 0.7861
Epoch: 9138, AUC: 0.7796, AP: 0.7861
Epoch: 9139, AUC: 0.7797, AP: 0.7862
E

Epoch: 9347, AUC: 0.7794, AP: 0.7864
Epoch: 9348, AUC: 0.7794, AP: 0.7864
Epoch: 9349, AUC: 0.7794, AP: 0.7864
Epoch: 9350, AUC: 0.7794, AP: 0.7864
Epoch: 9351, AUC: 0.7794, AP: 0.7864
Epoch: 9352, AUC: 0.7794, AP: 0.7864
Epoch: 9353, AUC: 0.7794, AP: 0.7864
Epoch: 9354, AUC: 0.7794, AP: 0.7864
Epoch: 9355, AUC: 0.7794, AP: 0.7864
Epoch: 9356, AUC: 0.7794, AP: 0.7863
Epoch: 9357, AUC: 0.7794, AP: 0.7864
Epoch: 9358, AUC: 0.7794, AP: 0.7864
Epoch: 9359, AUC: 0.7794, AP: 0.7864
Epoch: 9360, AUC: 0.7794, AP: 0.7864
Epoch: 9361, AUC: 0.7794, AP: 0.7864
Epoch: 9362, AUC: 0.7794, AP: 0.7864
Epoch: 9363, AUC: 0.7794, AP: 0.7864
Epoch: 9364, AUC: 0.7794, AP: 0.7864
Epoch: 9365, AUC: 0.7794, AP: 0.7864
Epoch: 9366, AUC: 0.7794, AP: 0.7864
Epoch: 9367, AUC: 0.7794, AP: 0.7864
Epoch: 9368, AUC: 0.7794, AP: 0.7864
Epoch: 9369, AUC: 0.7794, AP: 0.7864
Epoch: 9370, AUC: 0.7794, AP: 0.7864
Epoch: 9371, AUC: 0.7794, AP: 0.7864
Epoch: 9372, AUC: 0.7794, AP: 0.7864
Epoch: 9373, AUC: 0.7794, AP: 0.7864
E

Epoch: 9581, AUC: 0.7790, AP: 0.7862
Epoch: 9582, AUC: 0.7790, AP: 0.7862
Epoch: 9583, AUC: 0.7790, AP: 0.7862
Epoch: 9584, AUC: 0.7790, AP: 0.7862
Epoch: 9585, AUC: 0.7790, AP: 0.7862
Epoch: 9586, AUC: 0.7790, AP: 0.7862
Epoch: 9587, AUC: 0.7790, AP: 0.7861
Epoch: 9588, AUC: 0.7790, AP: 0.7861
Epoch: 9589, AUC: 0.7791, AP: 0.7861
Epoch: 9590, AUC: 0.7790, AP: 0.7862
Epoch: 9591, AUC: 0.7790, AP: 0.7862
Epoch: 9592, AUC: 0.7790, AP: 0.7862
Epoch: 9593, AUC: 0.7790, AP: 0.7862
Epoch: 9594, AUC: 0.7790, AP: 0.7861
Epoch: 9595, AUC: 0.7790, AP: 0.7862
Epoch: 9596, AUC: 0.7790, AP: 0.7862
Epoch: 9597, AUC: 0.7791, AP: 0.7862
Epoch: 9598, AUC: 0.7790, AP: 0.7862
Epoch: 9599, AUC: 0.7790, AP: 0.7862
Epoch: 9600, AUC: 0.7790, AP: 0.7862
Epoch: 9601, AUC: 0.7790, AP: 0.7862
Epoch: 9602, AUC: 0.7791, AP: 0.7862
Epoch: 9603, AUC: 0.7790, AP: 0.7862
Epoch: 9604, AUC: 0.7791, AP: 0.7862
Epoch: 9605, AUC: 0.7791, AP: 0.7862
Epoch: 9606, AUC: 0.7791, AP: 0.7862
Epoch: 9607, AUC: 0.7791, AP: 0.7862
E

Epoch: 9815, AUC: 0.7788, AP: 0.7863
Epoch: 9816, AUC: 0.7788, AP: 0.7864
Epoch: 9817, AUC: 0.7788, AP: 0.7863
Epoch: 9818, AUC: 0.7788, AP: 0.7863
Epoch: 9819, AUC: 0.7788, AP: 0.7863
Epoch: 9820, AUC: 0.7788, AP: 0.7864
Epoch: 9821, AUC: 0.7789, AP: 0.7864
Epoch: 9822, AUC: 0.7789, AP: 0.7864
Epoch: 9823, AUC: 0.7788, AP: 0.7864
Epoch: 9824, AUC: 0.7788, AP: 0.7863
Epoch: 9825, AUC: 0.7788, AP: 0.7863
Epoch: 9826, AUC: 0.7788, AP: 0.7863
Epoch: 9827, AUC: 0.7788, AP: 0.7863
Epoch: 9828, AUC: 0.7788, AP: 0.7863
Epoch: 9829, AUC: 0.7788, AP: 0.7863
Epoch: 9830, AUC: 0.7788, AP: 0.7863
Epoch: 9831, AUC: 0.7788, AP: 0.7863
Epoch: 9832, AUC: 0.7788, AP: 0.7864
Epoch: 9833, AUC: 0.7788, AP: 0.7864
Epoch: 9834, AUC: 0.7788, AP: 0.7864
Epoch: 9835, AUC: 0.7788, AP: 0.7864
Epoch: 9836, AUC: 0.7788, AP: 0.7864
Epoch: 9837, AUC: 0.7789, AP: 0.7864
Epoch: 9838, AUC: 0.7788, AP: 0.7864
Epoch: 9839, AUC: 0.7788, AP: 0.7864
Epoch: 9840, AUC: 0.7789, AP: 0.7865
Epoch: 9841, AUC: 0.7789, AP: 0.7865
E

# Varational Autoencoders

- KL divergence 


- Reparameterization trick



- The difference between is Autoencoder and VAE is Latent space concretely latter using the distribution in other words, more tendency that distribtuion

In [49]:
from torch_geometric.nn import VGAE

In [61]:
from optimizers import AvaGrad

In [62]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None # our task is unsupervised learning
data = train_test_split_edges(data)

class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) ## low-dimensional
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True) ## distriubtion of mu
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True) ## distirubiton of sigma
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

## AvaGrad

In [63]:
out_channels = 2
num_features = dataset.num_features
epochs = 5000

model = VGAE(VariationalGCNEncoder(num_features, out_channels)) # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

optimizer = AvaGrad(model.parameters(), lr=1e-2)

In [64]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    # if args.variational:
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss() # 쿨랙라이블러 발산을 활용한 distribution correction 
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
        
    return model.test(z, pos_edge_index, neg_edge_index)

In [66]:
wandb.init()

for epoch in range(1, epochs + 1):
    loss = train()
    
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    wandb.log({
        'test epoch' : epoch,
        'test accuracy' : auc,
        'test ap' : ap
    })
    
    print(f'Epoch: {epoch:03d}, AUC: {auc:4f}, AP: {ap:.4f}')

test epoch,5000
test accuracy,0.60738
test ap,0.63074
_runtime,2762
_timestamp,1629958017
_step,9999


test epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇█▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test accuracy,▇▇▆███▇███▇▇█▇▇▇▇▇▇▇▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test ap,▇▆▆██████████▇█▇▇▇▇█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████████████████████
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████████████████████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


Epoch: 001, AUC: 0.607381, AP: 0.6307
Epoch: 002, AUC: 0.607381, AP: 0.6307
Epoch: 003, AUC: 0.607386, AP: 0.6307
Epoch: 004, AUC: 0.607381, AP: 0.6307
Epoch: 005, AUC: 0.607374, AP: 0.6307
Epoch: 006, AUC: 0.607378, AP: 0.6307
Epoch: 007, AUC: 0.607381, AP: 0.6307
Epoch: 008, AUC: 0.607381, AP: 0.6307
Epoch: 009, AUC: 0.607381, AP: 0.6307
Epoch: 010, AUC: 0.607381, AP: 0.6307
Epoch: 011, AUC: 0.607381, AP: 0.6307
Epoch: 012, AUC: 0.607378, AP: 0.6307
Epoch: 013, AUC: 0.607381, AP: 0.6307
Epoch: 014, AUC: 0.607383, AP: 0.6307
Epoch: 015, AUC: 0.607386, AP: 0.6307
Epoch: 016, AUC: 0.607386, AP: 0.6307
Epoch: 017, AUC: 0.607386, AP: 0.6307
Epoch: 018, AUC: 0.607390, AP: 0.6307
Epoch: 019, AUC: 0.607400, AP: 0.6308
Epoch: 020, AUC: 0.607400, AP: 0.6308
Epoch: 021, AUC: 0.607395, AP: 0.6308
Epoch: 022, AUC: 0.607390, AP: 0.6308
Epoch: 023, AUC: 0.607400, AP: 0.6308
Epoch: 024, AUC: 0.607400, AP: 0.6308
Epoch: 025, AUC: 0.607402, AP: 0.6308
Epoch: 026, AUC: 0.607405, AP: 0.6308
Epoch: 027, 

Epoch: 217, AUC: 0.606989, AP: 0.6305
Epoch: 218, AUC: 0.606985, AP: 0.6305
Epoch: 219, AUC: 0.606989, AP: 0.6305
Epoch: 220, AUC: 0.606985, AP: 0.6305
Epoch: 221, AUC: 0.606989, AP: 0.6305
Epoch: 222, AUC: 0.606987, AP: 0.6304
Epoch: 223, AUC: 0.606989, AP: 0.6304
Epoch: 224, AUC: 0.606989, AP: 0.6304
Epoch: 225, AUC: 0.606989, AP: 0.6304
Epoch: 226, AUC: 0.606985, AP: 0.6304
Epoch: 227, AUC: 0.606985, AP: 0.6304
Epoch: 228, AUC: 0.606985, AP: 0.6304
Epoch: 229, AUC: 0.606975, AP: 0.6304
Epoch: 230, AUC: 0.606975, AP: 0.6304
Epoch: 231, AUC: 0.606970, AP: 0.6304
Epoch: 232, AUC: 0.606965, AP: 0.6304
Epoch: 233, AUC: 0.606965, AP: 0.6304
Epoch: 234, AUC: 0.606963, AP: 0.6304
Epoch: 235, AUC: 0.606956, AP: 0.6304
Epoch: 236, AUC: 0.606956, AP: 0.6304
Epoch: 237, AUC: 0.606956, AP: 0.6304
Epoch: 238, AUC: 0.606948, AP: 0.6304
Epoch: 239, AUC: 0.606946, AP: 0.6304
Epoch: 240, AUC: 0.606946, AP: 0.6304
Epoch: 241, AUC: 0.606946, AP: 0.6304
Epoch: 242, AUC: 0.606946, AP: 0.6304
Epoch: 243, 

Epoch: 437, AUC: 0.606738, AP: 0.6303
Epoch: 438, AUC: 0.606729, AP: 0.6303
Epoch: 439, AUC: 0.606729, AP: 0.6303
Epoch: 440, AUC: 0.606738, AP: 0.6303
Epoch: 441, AUC: 0.606738, AP: 0.6303
Epoch: 442, AUC: 0.606738, AP: 0.6303
Epoch: 443, AUC: 0.606738, AP: 0.6303
Epoch: 444, AUC: 0.606738, AP: 0.6303
Epoch: 445, AUC: 0.606738, AP: 0.6303
Epoch: 446, AUC: 0.606733, AP: 0.6303
Epoch: 447, AUC: 0.606733, AP: 0.6303
Epoch: 448, AUC: 0.606729, AP: 0.6303
Epoch: 449, AUC: 0.606729, AP: 0.6303
Epoch: 450, AUC: 0.606729, AP: 0.6303
Epoch: 451, AUC: 0.606724, AP: 0.6303
Epoch: 452, AUC: 0.606724, AP: 0.6303
Epoch: 453, AUC: 0.606724, AP: 0.6303
Epoch: 454, AUC: 0.606719, AP: 0.6303
Epoch: 455, AUC: 0.606719, AP: 0.6303
Epoch: 456, AUC: 0.606719, AP: 0.6303
Epoch: 457, AUC: 0.606719, AP: 0.6303
Epoch: 458, AUC: 0.606729, AP: 0.6303
Epoch: 459, AUC: 0.606724, AP: 0.6303
Epoch: 460, AUC: 0.606719, AP: 0.6303
Epoch: 461, AUC: 0.606719, AP: 0.6303
Epoch: 462, AUC: 0.606719, AP: 0.6303
Epoch: 463, 

...

Epoch: 4117, AUC: 0.605453, AP: 0.6293
Epoch: 4118, AUC: 0.605453, AP: 0.6293
Epoch: 4119, AUC: 0.605453, AP: 0.6293
Epoch: 4120, AUC: 0.605453, AP: 0.6293
Epoch: 4121, AUC: 0.605453, AP: 0.6293
Epoch: 4122, AUC: 0.605453, AP: 0.6293
Epoch: 4123, AUC: 0.605453, AP: 0.6293
Epoch: 4124, AUC: 0.605453, AP: 0.6293
Epoch: 4125, AUC: 0.605453, AP: 0.6293
Epoch: 4126, AUC: 0.605453, AP: 0.6293
Epoch: 4127, AUC: 0.605449, AP: 0.6293
Epoch: 4128, AUC: 0.605449, AP: 0.6293
Epoch: 4129, AUC: 0.605449, AP: 0.6293
Epoch: 4130, AUC: 0.605446, AP: 0.6293
Epoch: 4131, AUC: 0.605453, AP: 0.6293
Epoch: 4132, AUC: 0.605453, AP: 0.6293
Epoch: 4133, AUC: 0.605449, AP: 0.6293
Epoch: 4134, AUC: 0.605444, AP: 0.6293
Epoch: 4135, AUC: 0.605444, AP: 0.6293
Epoch: 4136, AUC: 0.605444, AP: 0.6293
Epoch: 4137, AUC: 0.605446, AP: 0.6293
Epoch: 4138, AUC: 0.605449, AP: 0.6293
Epoch: 4139, AUC: 0.605449, AP: 0.6293
Epoch: 4140, AUC: 0.605449, AP: 0.6293
Epoch: 4141, AUC: 0.605456, AP: 0.6293
Epoch: 4142, AUC: 0.60545

Epoch: 4337, AUC: 0.605478, AP: 0.6293
Epoch: 4338, AUC: 0.605473, AP: 0.6293
Epoch: 4339, AUC: 0.605473, AP: 0.6293
Epoch: 4340, AUC: 0.605473, AP: 0.6293
Epoch: 4341, AUC: 0.605475, AP: 0.6293
Epoch: 4342, AUC: 0.605468, AP: 0.6293
Epoch: 4343, AUC: 0.605468, AP: 0.6293
Epoch: 4344, AUC: 0.605468, AP: 0.6293
Epoch: 4345, AUC: 0.605468, AP: 0.6293
Epoch: 4346, AUC: 0.605468, AP: 0.6293
Epoch: 4347, AUC: 0.605468, AP: 0.6293
Epoch: 4348, AUC: 0.605468, AP: 0.6293
Epoch: 4349, AUC: 0.605468, AP: 0.6293
Epoch: 4350, AUC: 0.605473, AP: 0.6293
Epoch: 4351, AUC: 0.605478, AP: 0.6293
Epoch: 4352, AUC: 0.605473, AP: 0.6293
Epoch: 4353, AUC: 0.605473, AP: 0.6293
Epoch: 4354, AUC: 0.605473, AP: 0.6293
Epoch: 4355, AUC: 0.605473, AP: 0.6293
Epoch: 4356, AUC: 0.605473, AP: 0.6293
Epoch: 4357, AUC: 0.605468, AP: 0.6293
Epoch: 4358, AUC: 0.605468, AP: 0.6293
Epoch: 4359, AUC: 0.605478, AP: 0.6293
Epoch: 4360, AUC: 0.605478, AP: 0.6293
Epoch: 4361, AUC: 0.605482, AP: 0.6293
Epoch: 4362, AUC: 0.60548

Epoch: 4557, AUC: 0.605478, AP: 0.6293
Epoch: 4558, AUC: 0.605473, AP: 0.6293
Epoch: 4559, AUC: 0.605473, AP: 0.6293
Epoch: 4560, AUC: 0.605473, AP: 0.6293
Epoch: 4561, AUC: 0.605473, AP: 0.6293
Epoch: 4562, AUC: 0.605463, AP: 0.6293
Epoch: 4563, AUC: 0.605468, AP: 0.6293
Epoch: 4564, AUC: 0.605463, AP: 0.6293
Epoch: 4565, AUC: 0.605473, AP: 0.6293
Epoch: 4566, AUC: 0.605470, AP: 0.6293
Epoch: 4567, AUC: 0.605473, AP: 0.6293
Epoch: 4568, AUC: 0.605468, AP: 0.6293
Epoch: 4569, AUC: 0.605478, AP: 0.6293
Epoch: 4570, AUC: 0.605478, AP: 0.6293
Epoch: 4571, AUC: 0.605478, AP: 0.6293
Epoch: 4572, AUC: 0.605473, AP: 0.6293
Epoch: 4573, AUC: 0.605468, AP: 0.6293
Epoch: 4574, AUC: 0.605468, AP: 0.6293
Epoch: 4575, AUC: 0.605468, AP: 0.6293
Epoch: 4576, AUC: 0.605468, AP: 0.6293
Epoch: 4577, AUC: 0.605473, AP: 0.6293
Epoch: 4578, AUC: 0.605478, AP: 0.6293
Epoch: 4579, AUC: 0.605478, AP: 0.6293
Epoch: 4580, AUC: 0.605478, AP: 0.6293
Epoch: 4581, AUC: 0.605478, AP: 0.6293
Epoch: 4582, AUC: 0.60547

Epoch: 4775, AUC: 0.605531, AP: 0.6294
Epoch: 4776, AUC: 0.605531, AP: 0.6294
Epoch: 4777, AUC: 0.605531, AP: 0.6294
Epoch: 4778, AUC: 0.605531, AP: 0.6294
Epoch: 4779, AUC: 0.605531, AP: 0.6294
Epoch: 4780, AUC: 0.605550, AP: 0.6294
Epoch: 4781, AUC: 0.605552, AP: 0.6294
Epoch: 4782, AUC: 0.605543, AP: 0.6294
Epoch: 4783, AUC: 0.605545, AP: 0.6294
Epoch: 4784, AUC: 0.605543, AP: 0.6294
Epoch: 4785, AUC: 0.605540, AP: 0.6294
Epoch: 4786, AUC: 0.605540, AP: 0.6294
Epoch: 4787, AUC: 0.605540, AP: 0.6294
Epoch: 4788, AUC: 0.605540, AP: 0.6294
Epoch: 4789, AUC: 0.605540, AP: 0.6294
Epoch: 4790, AUC: 0.605543, AP: 0.6294
Epoch: 4791, AUC: 0.605540, AP: 0.6294
Epoch: 4792, AUC: 0.605540, AP: 0.6294
Epoch: 4793, AUC: 0.605543, AP: 0.6294
Epoch: 4794, AUC: 0.605545, AP: 0.6294
Epoch: 4795, AUC: 0.605545, AP: 0.6294
Epoch: 4796, AUC: 0.605552, AP: 0.6294
Epoch: 4797, AUC: 0.605555, AP: 0.6294
Epoch: 4798, AUC: 0.605560, AP: 0.6294
Epoch: 4799, AUC: 0.605555, AP: 0.6294
Epoch: 4800, AUC: 0.60556

Epoch: 4995, AUC: 0.605449, AP: 0.6292
Epoch: 4996, AUC: 0.605453, AP: 0.6292
Epoch: 4997, AUC: 0.605449, AP: 0.6292
Epoch: 4998, AUC: 0.605453, AP: 0.6292
Epoch: 4999, AUC: 0.605449, AP: 0.6292
Epoch: 5000, AUC: 0.605449, AP: 0.6292


In [70]:
!cat optimizers.py

import torch, math
from torch.optim.optimizer import Optimizer

class AvaGrad(Optimizer):
    def __init__(self, params, lr=0.1, betas=(0.9, 0.999), weight_decay=0, eps=0.1):
        defaults = dict(lr=lr, betas=betas, weight_decay=weight_decay, eps=eps, gamma=None)
        super(AvaGrad, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AvaGrad, self).__setstate__(state)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            gamma = group['gamma']
            squared_norm = 0.0
            num_params = 0.0

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                state = self.state[p]
                
                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p.data)
         

## AvaGradW

In [71]:
out_channels = 2
num_features = dataset.num_features
epochs = 5000

model = VGAE(VariationalGCNEncoder(num_features, out_channels)) # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

optimizer = AvaGradW(model.parameters(), lr=1e-2)

In [72]:
wandb.init(name='AvaGradW')

for epoch in range(1, epochs + 1):
    loss = train()
    
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    wandb.log({
        'test epoch' : epoch,
        'test accuracy' : auc,
        'test ap' : ap
    })
    
    print(f'Epoch: {epoch:03d}, AUC: {auc:4f}, AP: {ap:.4f}')

test epoch,5000
test accuracy,0.60545
test ap,0.62915
_runtime,100
_timestamp,1629958142
_step,4999


test epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test accuracy,█▇▆▆▅▅▅▄▄▃▃▃▂▂▂▂▁▁▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test ap,██▇▆▆▆▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▂▂▂▂▂▁▁▁▁▂▂▂▂▂▂▂▂▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


Epoch: 001, AUC: 0.573428, AP: 0.5625
Epoch: 002, AUC: 0.571211, AP: 0.5692
Epoch: 003, AUC: 0.574672, AP: 0.5750
Epoch: 004, AUC: 0.574054, AP: 0.5788
Epoch: 005, AUC: 0.574892, AP: 0.5841
Epoch: 006, AUC: 0.575001, AP: 0.5874
Epoch: 007, AUC: 0.575764, AP: 0.5907
Epoch: 008, AUC: 0.576464, AP: 0.5943
Epoch: 009, AUC: 0.577650, AP: 0.5996
Epoch: 010, AUC: 0.578534, AP: 0.6031
Epoch: 011, AUC: 0.579452, AP: 0.6063
Epoch: 012, AUC: 0.580302, AP: 0.6093
Epoch: 013, AUC: 0.581324, AP: 0.6128
Epoch: 014, AUC: 0.582712, AP: 0.6165
Epoch: 015, AUC: 0.583753, AP: 0.6189
Epoch: 016, AUC: 0.584693, AP: 0.6205
Epoch: 017, AUC: 0.585702, AP: 0.6221
Epoch: 018, AUC: 0.586948, AP: 0.6238
Epoch: 019, AUC: 0.587613, AP: 0.6248
Epoch: 020, AUC: 0.588499, AP: 0.6260
Epoch: 021, AUC: 0.589320, AP: 0.6275
Epoch: 022, AUC: 0.590197, AP: 0.6288
Epoch: 023, AUC: 0.591088, AP: 0.6297
Epoch: 024, AUC: 0.592069, AP: 0.6305
Epoch: 025, AUC: 0.592953, AP: 0.6313
Epoch: 026, AUC: 0.593566, AP: 0.6319
Epoch: 027, 

Epoch: 222, AUC: 0.616747, AP: 0.6419
Epoch: 223, AUC: 0.616771, AP: 0.6420
Epoch: 224, AUC: 0.616819, AP: 0.6420
Epoch: 225, AUC: 0.616797, AP: 0.6419
Epoch: 226, AUC: 0.616800, AP: 0.6419
Epoch: 227, AUC: 0.616848, AP: 0.6419
Epoch: 228, AUC: 0.616841, AP: 0.6419
Epoch: 229, AUC: 0.616831, AP: 0.6419
Epoch: 230, AUC: 0.616846, AP: 0.6419
Epoch: 231, AUC: 0.616812, AP: 0.6418
Epoch: 232, AUC: 0.616834, AP: 0.6418
Epoch: 233, AUC: 0.616810, AP: 0.6418
Epoch: 234, AUC: 0.616776, AP: 0.6418
Epoch: 235, AUC: 0.616740, AP: 0.6417
Epoch: 236, AUC: 0.616754, AP: 0.6417
Epoch: 237, AUC: 0.616776, AP: 0.6416
Epoch: 238, AUC: 0.616766, AP: 0.6416
Epoch: 239, AUC: 0.616773, AP: 0.6417
Epoch: 240, AUC: 0.616797, AP: 0.6417
Epoch: 241, AUC: 0.616778, AP: 0.6417
Epoch: 242, AUC: 0.616802, AP: 0.6417
Epoch: 243, AUC: 0.616846, AP: 0.6417
Epoch: 244, AUC: 0.616892, AP: 0.6418
Epoch: 245, AUC: 0.616916, AP: 0.6418
Epoch: 246, AUC: 0.616925, AP: 0.6418
Epoch: 247, AUC: 0.616986, AP: 0.6418
Epoch: 248, 

Epoch: 446, AUC: 0.616223, AP: 0.6400
Epoch: 447, AUC: 0.616225, AP: 0.6400
Epoch: 448, AUC: 0.616240, AP: 0.6400
Epoch: 449, AUC: 0.616213, AP: 0.6400
Epoch: 450, AUC: 0.616228, AP: 0.6399
Epoch: 451, AUC: 0.616215, AP: 0.6399
Epoch: 452, AUC: 0.616230, AP: 0.6399
Epoch: 453, AUC: 0.616244, AP: 0.6400
Epoch: 454, AUC: 0.616256, AP: 0.6400
Epoch: 455, AUC: 0.616269, AP: 0.6400
Epoch: 456, AUC: 0.616269, AP: 0.6399
Epoch: 457, AUC: 0.616278, AP: 0.6399
Epoch: 458, AUC: 0.616247, AP: 0.6399
Epoch: 459, AUC: 0.616244, AP: 0.6399
Epoch: 460, AUC: 0.616213, AP: 0.6399
Epoch: 461, AUC: 0.616194, AP: 0.6399
Epoch: 462, AUC: 0.616162, AP: 0.6399
Epoch: 463, AUC: 0.616126, AP: 0.6398
Epoch: 464, AUC: 0.616100, AP: 0.6398
Epoch: 465, AUC: 0.616133, AP: 0.6399
Epoch: 466, AUC: 0.616131, AP: 0.6399
Epoch: 467, AUC: 0.616078, AP: 0.6399
Epoch: 468, AUC: 0.616068, AP: 0.6399
Epoch: 469, AUC: 0.616058, AP: 0.6399
Epoch: 470, AUC: 0.616044, AP: 0.6398
Epoch: 471, AUC: 0.616015, AP: 0.6398
Epoch: 472, 

...

Epoch: 3936, AUC: 0.611554, AP: 0.6343
Epoch: 3937, AUC: 0.611559, AP: 0.6343
Epoch: 3938, AUC: 0.611547, AP: 0.6343
Epoch: 3939, AUC: 0.611540, AP: 0.6343
Epoch: 3940, AUC: 0.611540, AP: 0.6343
Epoch: 3941, AUC: 0.611530, AP: 0.6343
Epoch: 3942, AUC: 0.611532, AP: 0.6343
Epoch: 3943, AUC: 0.611535, AP: 0.6343
Epoch: 3944, AUC: 0.611525, AP: 0.6343
Epoch: 3945, AUC: 0.611520, AP: 0.6343
Epoch: 3946, AUC: 0.611520, AP: 0.6343
Epoch: 3947, AUC: 0.611523, AP: 0.6343
Epoch: 3948, AUC: 0.611520, AP: 0.6343
Epoch: 3949, AUC: 0.611511, AP: 0.6343
Epoch: 3950, AUC: 0.611506, AP: 0.6343
Epoch: 3951, AUC: 0.611496, AP: 0.6342
Epoch: 3952, AUC: 0.611496, AP: 0.6342
Epoch: 3953, AUC: 0.611491, AP: 0.6342
Epoch: 3954, AUC: 0.611484, AP: 0.6342
Epoch: 3955, AUC: 0.611477, AP: 0.6342
Epoch: 3956, AUC: 0.611484, AP: 0.6342
Epoch: 3957, AUC: 0.611487, AP: 0.6342
Epoch: 3958, AUC: 0.611487, AP: 0.6342
Epoch: 3959, AUC: 0.611484, AP: 0.6342
Epoch: 3960, AUC: 0.611477, AP: 0.6342
Epoch: 3961, AUC: 0.61147

Epoch: 4157, AUC: 0.611182, AP: 0.6338
Epoch: 4158, AUC: 0.611182, AP: 0.6338
Epoch: 4159, AUC: 0.611182, AP: 0.6338
Epoch: 4160, AUC: 0.611182, AP: 0.6338
Epoch: 4161, AUC: 0.611177, AP: 0.6338
Epoch: 4162, AUC: 0.611177, AP: 0.6338
Epoch: 4163, AUC: 0.611177, AP: 0.6338
Epoch: 4164, AUC: 0.611177, AP: 0.6338
Epoch: 4165, AUC: 0.611175, AP: 0.6338
Epoch: 4166, AUC: 0.611177, AP: 0.6338
Epoch: 4167, AUC: 0.611177, AP: 0.6338
Epoch: 4168, AUC: 0.611173, AP: 0.6338
Epoch: 4169, AUC: 0.611177, AP: 0.6338
Epoch: 4170, AUC: 0.611177, AP: 0.6338
Epoch: 4171, AUC: 0.611173, AP: 0.6338
Epoch: 4172, AUC: 0.611177, AP: 0.6338
Epoch: 4173, AUC: 0.611168, AP: 0.6338
Epoch: 4174, AUC: 0.611158, AP: 0.6337
Epoch: 4175, AUC: 0.611163, AP: 0.6337
Epoch: 4176, AUC: 0.611163, AP: 0.6337
Epoch: 4177, AUC: 0.611158, AP: 0.6337
Epoch: 4178, AUC: 0.611156, AP: 0.6337
Epoch: 4179, AUC: 0.611148, AP: 0.6337
Epoch: 4180, AUC: 0.611148, AP: 0.6337
Epoch: 4181, AUC: 0.611148, AP: 0.6337
Epoch: 4182, AUC: 0.61114

Epoch: 4377, AUC: 0.610762, AP: 0.6333
Epoch: 4378, AUC: 0.610757, AP: 0.6333
Epoch: 4379, AUC: 0.610757, AP: 0.6333
Epoch: 4380, AUC: 0.610757, AP: 0.6333
Epoch: 4381, AUC: 0.610752, AP: 0.6333
Epoch: 4382, AUC: 0.610752, AP: 0.6333
Epoch: 4383, AUC: 0.610752, AP: 0.6333
Epoch: 4384, AUC: 0.610747, AP: 0.6333
Epoch: 4385, AUC: 0.610738, AP: 0.6333
Epoch: 4386, AUC: 0.610731, AP: 0.6333
Epoch: 4387, AUC: 0.610733, AP: 0.6333
Epoch: 4388, AUC: 0.610723, AP: 0.6333
Epoch: 4389, AUC: 0.610723, AP: 0.6333
Epoch: 4390, AUC: 0.610728, AP: 0.6333
Epoch: 4391, AUC: 0.610726, AP: 0.6333
Epoch: 4392, AUC: 0.610726, AP: 0.6333
Epoch: 4393, AUC: 0.610726, AP: 0.6333
Epoch: 4394, AUC: 0.610719, AP: 0.6333
Epoch: 4395, AUC: 0.610719, AP: 0.6333
Epoch: 4396, AUC: 0.610719, AP: 0.6333
Epoch: 4397, AUC: 0.610721, AP: 0.6333
Epoch: 4398, AUC: 0.610731, AP: 0.6333
Epoch: 4399, AUC: 0.610733, AP: 0.6333
Epoch: 4400, AUC: 0.610728, AP: 0.6333
Epoch: 4401, AUC: 0.610728, AP: 0.6333
Epoch: 4402, AUC: 0.61072

Epoch: 4597, AUC: 0.610279, AP: 0.6329
Epoch: 4598, AUC: 0.610279, AP: 0.6329
Epoch: 4599, AUC: 0.610274, AP: 0.6329
Epoch: 4600, AUC: 0.610264, AP: 0.6329
Epoch: 4601, AUC: 0.610260, AP: 0.6329
Epoch: 4602, AUC: 0.610260, AP: 0.6329
Epoch: 4603, AUC: 0.610260, AP: 0.6329
Epoch: 4604, AUC: 0.610255, AP: 0.6329
Epoch: 4605, AUC: 0.610255, AP: 0.6329
Epoch: 4606, AUC: 0.610255, AP: 0.6329
Epoch: 4607, AUC: 0.610255, AP: 0.6329
Epoch: 4608, AUC: 0.610255, AP: 0.6329
Epoch: 4609, AUC: 0.610260, AP: 0.6329
Epoch: 4610, AUC: 0.610257, AP: 0.6329
Epoch: 4611, AUC: 0.610245, AP: 0.6328
Epoch: 4612, AUC: 0.610233, AP: 0.6328
Epoch: 4613, AUC: 0.610235, AP: 0.6328
Epoch: 4614, AUC: 0.610240, AP: 0.6328
Epoch: 4615, AUC: 0.610223, AP: 0.6328
Epoch: 4616, AUC: 0.610221, AP: 0.6328
Epoch: 4617, AUC: 0.610216, AP: 0.6328
Epoch: 4618, AUC: 0.610214, AP: 0.6328
Epoch: 4619, AUC: 0.610211, AP: 0.6328
Epoch: 4620, AUC: 0.610211, AP: 0.6328
Epoch: 4621, AUC: 0.610206, AP: 0.6328
Epoch: 4622, AUC: 0.61020

Epoch: 4817, AUC: 0.610033, AP: 0.6327
Epoch: 4818, AUC: 0.610033, AP: 0.6327
Epoch: 4819, AUC: 0.610033, AP: 0.6327
Epoch: 4820, AUC: 0.610037, AP: 0.6327
Epoch: 4821, AUC: 0.610037, AP: 0.6327
Epoch: 4822, AUC: 0.610037, AP: 0.6327
Epoch: 4823, AUC: 0.610037, AP: 0.6327
Epoch: 4824, AUC: 0.610037, AP: 0.6327
Epoch: 4825, AUC: 0.610037, AP: 0.6327
Epoch: 4826, AUC: 0.610045, AP: 0.6327
Epoch: 4827, AUC: 0.610047, AP: 0.6327
Epoch: 4828, AUC: 0.610047, AP: 0.6327
Epoch: 4829, AUC: 0.610047, AP: 0.6327
Epoch: 4830, AUC: 0.610047, AP: 0.6327
Epoch: 4831, AUC: 0.610047, AP: 0.6327
Epoch: 4832, AUC: 0.610047, AP: 0.6327
Epoch: 4833, AUC: 0.610037, AP: 0.6327
Epoch: 4834, AUC: 0.610037, AP: 0.6327
Epoch: 4835, AUC: 0.610042, AP: 0.6327
Epoch: 4836, AUC: 0.610042, AP: 0.6327
Epoch: 4837, AUC: 0.610042, AP: 0.6327
Epoch: 4838, AUC: 0.610040, AP: 0.6327
Epoch: 4839, AUC: 0.610045, AP: 0.6327
Epoch: 4840, AUC: 0.610042, AP: 0.6327
Epoch: 4841, AUC: 0.610042, AP: 0.6327
Epoch: 4842, AUC: 0.61003

## Adam

In [76]:
out_channels = 2
num_features = dataset.num_features
epochs = 5000

model = VGAE(VariationalGCNEncoder(num_features, out_channels)) # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [77]:
wandb.init(name='Adam')

for epoch in range(1, epochs + 1):
    loss = train()
    
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    wandb.log({
        'test epoch' : epoch,
        'test accuracy' : auc,
        'test ap' : ap
    })
    
    print(f'Epoch: {epoch:03d}, AUC: {auc:4f}, AP: {ap:.4f}')

test epoch,5000
test accuracy,0.60977
test ap,0.63248
_runtime,100
_timestamp,1629958336
_step,4999


test epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test accuracy,▁▆██▆▆▆▆▆▆▆▆▇▇▇▇▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂
test ap,▇██▇▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


Epoch: 001, AUC: 0.590308, AP: 0.6186
Epoch: 002, AUC: 0.606789, AP: 0.6380
Epoch: 003, AUC: 0.613486, AP: 0.6434
Epoch: 004, AUC: 0.616706, AP: 0.6446
Epoch: 005, AUC: 0.618111, AP: 0.6449
Epoch: 006, AUC: 0.619010, AP: 0.6450
Epoch: 007, AUC: 0.619633, AP: 0.6452
Epoch: 008, AUC: 0.619824, AP: 0.6451
Epoch: 009, AUC: 0.619466, AP: 0.6445
Epoch: 010, AUC: 0.619710, AP: 0.6442
Epoch: 011, AUC: 0.619901, AP: 0.6443
Epoch: 012, AUC: 0.620039, AP: 0.6440
Epoch: 013, AUC: 0.619932, AP: 0.6437
Epoch: 014, AUC: 0.619319, AP: 0.6431
Epoch: 015, AUC: 0.618623, AP: 0.6419
Epoch: 016, AUC: 0.617544, AP: 0.6410
Epoch: 017, AUC: 0.616211, AP: 0.6393
Epoch: 018, AUC: 0.615860, AP: 0.6391
Epoch: 019, AUC: 0.615885, AP: 0.6392
Epoch: 020, AUC: 0.615435, AP: 0.6387
Epoch: 021, AUC: 0.616058, AP: 0.6393
Epoch: 022, AUC: 0.617124, AP: 0.6401
Epoch: 023, AUC: 0.618645, AP: 0.6416
Epoch: 024, AUC: 0.620389, AP: 0.6432
Epoch: 025, AUC: 0.621816, AP: 0.6442
Epoch: 026, AUC: 0.623611, AP: 0.6460
Epoch: 027, 

Epoch: 228, AUC: 0.790697, AP: 0.7986
Epoch: 229, AUC: 0.791170, AP: 0.7989
Epoch: 230, AUC: 0.792035, AP: 0.7999
Epoch: 231, AUC: 0.792296, AP: 0.8001
Epoch: 232, AUC: 0.790301, AP: 0.7982
Epoch: 233, AUC: 0.787948, AP: 0.7958
Epoch: 234, AUC: 0.787006, AP: 0.7948
Epoch: 235, AUC: 0.788586, AP: 0.7968
Epoch: 236, AUC: 0.790489, AP: 0.7982
Epoch: 237, AUC: 0.791929, AP: 0.7998
Epoch: 238, AUC: 0.791383, AP: 0.7992
Epoch: 239, AUC: 0.789673, AP: 0.7974
Epoch: 240, AUC: 0.788533, AP: 0.7967
Epoch: 241, AUC: 0.788272, AP: 0.7965
Epoch: 242, AUC: 0.788513, AP: 0.7968
Epoch: 243, AUC: 0.789161, AP: 0.7972
Epoch: 244, AUC: 0.789576, AP: 0.7975
Epoch: 245, AUC: 0.789320, AP: 0.7974
Epoch: 246, AUC: 0.788929, AP: 0.7970
Epoch: 247, AUC: 0.788499, AP: 0.7968
Epoch: 248, AUC: 0.788190, AP: 0.7965
Epoch: 249, AUC: 0.787799, AP: 0.7963
Epoch: 250, AUC: 0.787948, AP: 0.7964
Epoch: 251, AUC: 0.788939, AP: 0.7971
Epoch: 252, AUC: 0.788963, AP: 0.7972
Epoch: 253, AUC: 0.788373, AP: 0.7966
Epoch: 254, 

Epoch: 454, AUC: 0.772718, AP: 0.7863
Epoch: 455, AUC: 0.772569, AP: 0.7861
Epoch: 456, AUC: 0.772124, AP: 0.7853
Epoch: 457, AUC: 0.771375, AP: 0.7846
Epoch: 458, AUC: 0.771139, AP: 0.7844
Epoch: 459, AUC: 0.772008, AP: 0.7854
Epoch: 460, AUC: 0.772559, AP: 0.7860
Epoch: 461, AUC: 0.773018, AP: 0.7866
Epoch: 462, AUC: 0.772438, AP: 0.7858
Epoch: 463, AUC: 0.771028, AP: 0.7844
Epoch: 464, AUC: 0.771801, AP: 0.7854
Epoch: 465, AUC: 0.772916, AP: 0.7868
Epoch: 466, AUC: 0.773665, AP: 0.7879
Epoch: 467, AUC: 0.773539, AP: 0.7878
Epoch: 468, AUC: 0.771554, AP: 0.7851
Epoch: 469, AUC: 0.768965, AP: 0.7815
Epoch: 470, AUC: 0.769893, AP: 0.7829
Epoch: 471, AUC: 0.772274, AP: 0.7862
Epoch: 472, AUC: 0.773675, AP: 0.7879
Epoch: 473, AUC: 0.774056, AP: 0.7884
Epoch: 474, AUC: 0.772201, AP: 0.7865
Epoch: 475, AUC: 0.770738, AP: 0.7843
Epoch: 476, AUC: 0.770105, AP: 0.7834
Epoch: 477, AUC: 0.771187, AP: 0.7849
Epoch: 478, AUC: 0.772419, AP: 0.7870
Epoch: 479, AUC: 0.772805, AP: 0.7876
Epoch: 480, 

...

Epoch: 4127, AUC: 0.809289, AP: 0.8296
Epoch: 4128, AUC: 0.809598, AP: 0.8298
Epoch: 4129, AUC: 0.809748, AP: 0.8299
Epoch: 4130, AUC: 0.809603, AP: 0.8297
Epoch: 4131, AUC: 0.809081, AP: 0.8293
Epoch: 4132, AUC: 0.808801, AP: 0.8287
Epoch: 4133, AUC: 0.809023, AP: 0.8288
Epoch: 4134, AUC: 0.810023, AP: 0.8298
Epoch: 4135, AUC: 0.810405, AP: 0.8303
Epoch: 4136, AUC: 0.810052, AP: 0.8304
Epoch: 4137, AUC: 0.809950, AP: 0.8303
Epoch: 4138, AUC: 0.809598, AP: 0.8299
Epoch: 4139, AUC: 0.808738, AP: 0.8288
Epoch: 4140, AUC: 0.808530, AP: 0.8286
Epoch: 4141, AUC: 0.809458, AP: 0.8297
Epoch: 4142, AUC: 0.810293, AP: 0.8302
Epoch: 4143, AUC: 0.810506, AP: 0.8304
Epoch: 4144, AUC: 0.810139, AP: 0.8299
Epoch: 4145, AUC: 0.809144, AP: 0.8296
Epoch: 4146, AUC: 0.808562, AP: 0.8291
Epoch: 4147, AUC: 0.808033, AP: 0.8282
Epoch: 4148, AUC: 0.808062, AP: 0.8286
Epoch: 4149, AUC: 0.808622, AP: 0.8295
Epoch: 4150, AUC: 0.809352, AP: 0.8300
Epoch: 4151, AUC: 0.809583, AP: 0.8300
Epoch: 4152, AUC: 0.80916

Epoch: 4339, AUC: 0.808009, AP: 0.8287
Epoch: 4340, AUC: 0.808835, AP: 0.8296
Epoch: 4341, AUC: 0.808555, AP: 0.8295
Epoch: 4342, AUC: 0.807738, AP: 0.8290
Epoch: 4343, AUC: 0.807130, AP: 0.8285
Epoch: 4344, AUC: 0.807188, AP: 0.8282
Epoch: 4345, AUC: 0.807729, AP: 0.8280
Epoch: 4346, AUC: 0.807912, AP: 0.8284
Epoch: 4347, AUC: 0.807700, AP: 0.8286
Epoch: 4348, AUC: 0.806714, AP: 0.8283
Epoch: 4349, AUC: 0.805951, AP: 0.8280
Epoch: 4350, AUC: 0.806989, AP: 0.8285
Epoch: 4351, AUC: 0.807207, AP: 0.8285
Epoch: 4352, AUC: 0.806941, AP: 0.8281
Epoch: 4353, AUC: 0.806366, AP: 0.8274
Epoch: 4354, AUC: 0.805729, AP: 0.8274
Epoch: 4355, AUC: 0.805787, AP: 0.8280
Epoch: 4356, AUC: 0.806231, AP: 0.8278
Epoch: 4357, AUC: 0.806849, AP: 0.8281
Epoch: 4358, AUC: 0.807410, AP: 0.8284
Epoch: 4359, AUC: 0.806830, AP: 0.8281
Epoch: 4360, AUC: 0.806130, AP: 0.8277
Epoch: 4361, AUC: 0.805714, AP: 0.8276
Epoch: 4362, AUC: 0.805874, AP: 0.8277
Epoch: 4363, AUC: 0.806444, AP: 0.8279
Epoch: 4364, AUC: 0.80739

Epoch: 4551, AUC: 0.812675, AP: 0.8325
Epoch: 4552, AUC: 0.812428, AP: 0.8321
Epoch: 4553, AUC: 0.812554, AP: 0.8318
Epoch: 4554, AUC: 0.812829, AP: 0.8321
Epoch: 4555, AUC: 0.812578, AP: 0.8320
Epoch: 4556, AUC: 0.812245, AP: 0.8319
Epoch: 4557, AUC: 0.812057, AP: 0.8318
Epoch: 4558, AUC: 0.812419, AP: 0.8321
Epoch: 4559, AUC: 0.813283, AP: 0.8324
Epoch: 4560, AUC: 0.812940, AP: 0.8320
Epoch: 4561, AUC: 0.812895, AP: 0.8319
Epoch: 4562, AUC: 0.812085, AP: 0.8315
Epoch: 4563, AUC: 0.812110, AP: 0.8320
Epoch: 4564, AUC: 0.812395, AP: 0.8324
Epoch: 4565, AUC: 0.812622, AP: 0.8325
Epoch: 4566, AUC: 0.812216, AP: 0.8320
Epoch: 4567, AUC: 0.812201, AP: 0.8314
Epoch: 4568, AUC: 0.812776, AP: 0.8312
Epoch: 4569, AUC: 0.813288, AP: 0.8321
Epoch: 4570, AUC: 0.813588, AP: 0.8329
Epoch: 4571, AUC: 0.812916, AP: 0.8327
Epoch: 4572, AUC: 0.812375, AP: 0.8322
Epoch: 4573, AUC: 0.812844, AP: 0.8324
Epoch: 4574, AUC: 0.813583, AP: 0.8326
Epoch: 4575, AUC: 0.814525, AP: 0.8331
Epoch: 4576, AUC: 0.81507

Epoch: 4762, AUC: 0.808994, AP: 0.8289
Epoch: 4763, AUC: 0.808835, AP: 0.8290
Epoch: 4764, AUC: 0.809579, AP: 0.8294
Epoch: 4765, AUC: 0.810187, AP: 0.8293
Epoch: 4766, AUC: 0.810260, AP: 0.8292
Epoch: 4767, AUC: 0.809559, AP: 0.8290
Epoch: 4768, AUC: 0.808501, AP: 0.8288
Epoch: 4769, AUC: 0.808115, AP: 0.8284
Epoch: 4770, AUC: 0.808347, AP: 0.8286
Epoch: 4771, AUC: 0.809236, AP: 0.8290
Epoch: 4772, AUC: 0.810960, AP: 0.8297
Epoch: 4773, AUC: 0.811974, AP: 0.8304
Epoch: 4774, AUC: 0.810921, AP: 0.8300
Epoch: 4775, AUC: 0.809622, AP: 0.8295
Epoch: 4776, AUC: 0.809091, AP: 0.8293
Epoch: 4777, AUC: 0.809091, AP: 0.8290
Epoch: 4778, AUC: 0.808868, AP: 0.8287
Epoch: 4779, AUC: 0.809608, AP: 0.8290
Epoch: 4780, AUC: 0.810100, AP: 0.8295
Epoch: 4781, AUC: 0.810636, AP: 0.8301
Epoch: 4782, AUC: 0.810400, AP: 0.8301
Epoch: 4783, AUC: 0.810245, AP: 0.8299
Epoch: 4784, AUC: 0.810124, AP: 0.8297
Epoch: 4785, AUC: 0.809859, AP: 0.8294
Epoch: 4786, AUC: 0.809583, AP: 0.8292
Epoch: 4787, AUC: 0.80878

Epoch: 4977, AUC: 0.810714, AP: 0.8311
Epoch: 4978, AUC: 0.810289, AP: 0.8307
Epoch: 4979, AUC: 0.810810, AP: 0.8309
Epoch: 4980, AUC: 0.812607, AP: 0.8321
Epoch: 4981, AUC: 0.813327, AP: 0.8326
Epoch: 4982, AUC: 0.812965, AP: 0.8324
Epoch: 4983, AUC: 0.811738, AP: 0.8318
Epoch: 4984, AUC: 0.810839, AP: 0.8312
Epoch: 4985, AUC: 0.809704, AP: 0.8305
Epoch: 4986, AUC: 0.809680, AP: 0.8304
Epoch: 4987, AUC: 0.810250, AP: 0.8307
Epoch: 4988, AUC: 0.811264, AP: 0.8311
Epoch: 4989, AUC: 0.811786, AP: 0.8316
Epoch: 4990, AUC: 0.812066, AP: 0.8318
Epoch: 4991, AUC: 0.811892, AP: 0.8318
Epoch: 4992, AUC: 0.811293, AP: 0.8309
Epoch: 4993, AUC: 0.810298, AP: 0.8296
Epoch: 4994, AUC: 0.810477, AP: 0.8299
Epoch: 4995, AUC: 0.811931, AP: 0.8317
Epoch: 4996, AUC: 0.812694, AP: 0.8324
Epoch: 4997, AUC: 0.812728, AP: 0.8324
Epoch: 4998, AUC: 0.812342, AP: 0.8316
Epoch: 4999, AUC: 0.811004, AP: 0.8304
Epoch: 5000, AUC: 0.810354, AP: 0.8292
